In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
from pyspark.sql import SparkSession
my_spark = SparkSession.builder.master("spark://127.0.0.1:7077").appName("hadoop").getOrCreate()

In [ ]:
sqlCon = SQLContext(spark)

In [ ]:
df2 = sqlCon.read.option("header",False) \
     .option("inferScema", True) \
     .csv("hdfs:///ass1/pullreq_events.csv") \

In [ ]:
df3 = df2.withColumnRenamed("_c0","pull_requestid") \
    .withColumnRenamed("_c1","author") \
    .withColumnRenamed("_c2", "event") \
    .withColumnRenamed("_c3", "time")

In [ ]:
spark.sql("drop view pr")

DataFrame[]

In [ ]:
df3.createTempView("pr")

In [ ]:
%%time
my_spark.sql(" select a.name,b.week,b.countscore \
from(select hello.name, max(events) even, hello.week \
from (select author as name,date_trunc('week',time) week, count(event) events \
from pr where event='discussed' \
group by date_trunc('week',time), author) as hello \
group by hello.week,hello.name \
order by week,even desc)as a, \
(select hello.week, max(events) countscore \
from (select date_trunc('week',time) week, count(event) events \
from pr where event='discussed' \
group by date_trunc('week',time), author) as hello \
group by hello.week)as b \
where a.week=b.week and a.even=b.countscore").show()

+------------+-------------------+----------+
|        name|               week|countscore|
+------------+-------------------+----------+
|       mikel|2010-09-06 00:00:00|        16|
|       mikel|2010-09-13 00:00:00|         6|
|   josevalim|2010-09-20 00:00:00|         9|
|   josevalim|2010-09-27 00:00:00|         6|
|   josevalim|2010-10-04 00:00:00|        12|
|   josevalim|2010-10-11 00:00:00|         6|
|    krekoten|2010-10-18 00:00:00|         4|
|         fxn|2010-10-25 00:00:00|         1|
|  spastorino|2010-11-01 00:00:00|         2|
|        rsim|2010-11-01 00:00:00|         2|
|   josevalim|2010-11-08 00:00:00|         6|
|franckverrot|2010-11-15 00:00:00|         3|
|   josevalim|2010-11-15 00:00:00|         3|
|  tenderlove|2010-11-22 00:00:00|         6|
|   josevalim|2010-11-29 00:00:00|         4|
|      drogus|2010-12-06 00:00:00|         2|
|   josevalim|2010-12-13 00:00:00|         7|
|         dhh|2010-12-20 00:00:00|         5|
|      jeremy|2011-01-03 00:00:00|

In [ ]:
%%time
my_spark.sql("select a.name,b.monthnumber,b.countscore from(select hello.name, max(events) even, hello.monthnumber   from (select author as name,EXTRACT(MONTH from time) monthnumber, count(event) events   from pr where event='discussed'  group by EXTRACT(MONTH from time), author) as hello   group by hello.monthnumber,hello.name  order by monthnumber,even desc)as a,  (select hello.monthnumber, max(events) countscore   from (select EXTRACT(MONTH from time) monthnumber, count(event) events   from pr where event='discussed'  group by EXTRACT(MONTH from time), author) as hello  group by hello.monthnumber)as b  where a.monthnumber=b.monthnumber and a.even=b.countscore").show()

+------------+-----------+----------+
|        name|monthnumber|countscore|
+------------+-----------+----------+
|rafaelfranca|          1|       828|
|rafaelfranca|          2|       555|
|rafaelfranca|          3|       580|
|rafaelfranca|          4|       758|
|rafaelfranca|          5|       915|
|rafaelfranca|          6|       582|
|rafaelfranca|          7|       579|
|rafaelfranca|          8|       651|
|rafaelfranca|          9|       585|
|rafaelfranca|         10|       667|
|rafaelfranca|         11|       590|
|   rails-bot|         12|       546|
+------------+-----------+----------+

CPU times: user 3.2 ms, sys: 12.8 ms, total: 16 ms
Wall time: 13.1 s


In [ ]:
%%time
#1b
my_spark.sql("select event,date(time),count(*) from pr group by event,date(time) having event='discussed' order by date(time)").show()

+---------+----------+--------+
|    event|      time|count(1)|
+---------+----------+--------+
|discussed|2010-09-09|       6|
|discussed|2010-09-10|      10|
|discussed|2010-09-11|      13|
|discussed|2010-09-12|       5|
|discussed|2010-09-13|       7|
|discussed|2010-09-14|       1|
|discussed|2010-09-15|       3|
|discussed|2010-09-16|       2|
|discussed|2010-09-17|       1|
|discussed|2010-09-21|       6|
|discussed|2010-09-22|       3|
|discussed|2010-09-23|       5|
|discussed|2010-09-24|       3|
|discussed|2010-09-25|       5|
|discussed|2010-09-27|       3|
|discussed|2010-09-29|       2|
|discussed|2010-09-30|       3|
|discussed|2010-10-01|       2|
|discussed|2010-10-04|       8|
|discussed|2010-10-06|      15|
+---------+----------+--------+
only showing top 20 rows

CPU times: user 1.39 ms, sys: 12.1 ms, total: 13.5 ms
Wall time: 2.03 s


In [ ]:
%%time
#5
my_spark.sql("select EXTRACT(MONTH FROM time) mon, count(event) from pr where event='merged' and extract(year from time)=2010 group by mon").show()

+---+------------+
|mon|count(event)|
+---+------------+
+---+------------+

CPU times: user 0 ns, sys: 9.8 ms, total: 9.8 ms
Wall time: 2.02 s


In [ ]:
%%time
#1
my_spark.sql("select event,date(time),count(*) from pr group by event,date(time) having event='opened' order by date(time)").show()

+------+----------+--------+
| event|      time|count(1)|
+------+----------+--------+
|opened|2010-09-02|       2|
|opened|2010-09-06|       1|
|opened|2010-09-08|       1|
|opened|2010-09-09|       4|
|opened|2010-09-10|       3|
|opened|2010-09-11|       3|
|opened|2010-09-12|       3|
|opened|2010-09-13|       3|
|opened|2010-09-15|       2|
|opened|2010-09-16|       2|
|opened|2010-09-18|       6|
|opened|2010-09-19|       4|
|opened|2010-09-20|       2|
|opened|2010-09-22|       1|
|opened|2010-09-23|       4|
|opened|2010-09-24|       5|
|opened|2010-09-25|       5|
|opened|2010-09-27|       4|
|opened|2010-09-28|       2|
|opened|2010-09-29|       2|
+------+----------+--------+
only showing top 20 rows

CPU times: user 5.03 ms, sys: 2.05 ms, total: 7.07 ms
Wall time: 2.18 s


In [ ]:
%%time
#6
my_spark.sql("select date(time), count(pull_requestId) from pr group by date(time) order by date(time)").show()

+----------+---------------------+
|      time|count(pull_requestId)|
+----------+---------------------+
|2010-09-02|                    2|
|2010-09-06|                    1|
|2010-09-08|                    1|
|2010-09-09|                   10|
|2010-09-10|                   13|
|2010-09-11|                   16|
|2010-09-12|                    8|
|2010-09-13|                   10|
|2010-09-14|                    1|
|2010-09-15|                    5|
|2010-09-16|                    4|
|2010-09-17|                    1|
|2010-09-18|                    6|
|2010-09-19|                    4|
|2010-09-20|                    2|
|2010-09-21|                    6|
|2010-09-22|                    4|
|2010-09-23|                    9|
|2010-09-24|                    8|
|2010-09-25|                   10|
+----------+---------------------+
only showing top 20 rows

CPU times: user 5.82 ms, sys: 2.36 ms, total: 8.19 ms
Wall time: 2.13 s


In [ ]:
%%time
#7
my_spark.sql("select author, count(event) events from pr where event='opened' and extract(year from time)=2011 group by author order by events desc limit 1").show()

+-------+------+
| author|events|
+-------+------+
|arunagw|   228|
+-------+------+

CPU times: user 4.3 ms, sys: 1.76 ms, total: 6.06 ms
Wall time: 2.54 s


In [ ]:
%%time
#3
my_spark.sql("select a.name,b.week,b.countscore   from(select hello.name, max(events) even, hello.week  from (select author as name,date_trunc('week',time) week, count(event) events  from pr where event='discussed'   group by date_trunc('week',time), author) as hello  group by hello.week,hello.name  order by week,even desc)as a,   (select hello.week, max(events) countscore  from (select date_trunc('week',time) week, count(event) events  from pr where event='discussed'   group by date_trunc('week',time), author) as hello  group by hello.week)as b  where a.week=b.week and a.even=b.countscore").show()

+------------+-------------------+----------+
|        name|               week|countscore|
+------------+-------------------+----------+
|       mikel|2010-09-06 00:00:00|        16|
|       mikel|2010-09-13 00:00:00|         6|
|   josevalim|2010-09-20 00:00:00|         9|
|   josevalim|2010-09-27 00:00:00|         6|
|   josevalim|2010-10-04 00:00:00|        12|
|   josevalim|2010-10-11 00:00:00|         6|
|    krekoten|2010-10-18 00:00:00|         4|
|         fxn|2010-10-25 00:00:00|         1|
|  spastorino|2010-11-01 00:00:00|         2|
|        rsim|2010-11-01 00:00:00|         2|
|   josevalim|2010-11-08 00:00:00|         6|
|franckverrot|2010-11-15 00:00:00|         3|
|   josevalim|2010-11-15 00:00:00|         3|
|  tenderlove|2010-11-22 00:00:00|         6|
|   josevalim|2010-11-29 00:00:00|         4|
|      drogus|2010-12-06 00:00:00|         2|
|   josevalim|2010-12-13 00:00:00|         7|
|         dhh|2010-12-20 00:00:00|         5|
|      jeremy|2011-01-03 00:00:00|